In [8]:
import pandas
import mmengine
# info = pandas.read_pickle('data/kitti-360/kitti360_infos_all.pkl')

In [9]:
from projects.detr3d.custom_concat_dataset import CustomNusc
from projects.configs.mono_ablation.metainfo import nusc_val
import copy
from mmdet3d.utils import register_all_modules, replace_ceph_backend

In [10]:
register_all_modules()
cfg = copy.deepcopy(nusc_val)
# cfg['ann_file'] = 'nuscenes_infos_val.pkl'
cfg['ann_file'] = 'nuscenes_infos_train_mono_front_bevdet.pkl'
cfg.pop('type')
cfg.pop('load_interval',None)
dataset = CustomNusc(**cfg)

08/08 18:53:31 - mmengine - INFO - ------------------------------
08/08 18:53:31 - mmengine - INFO - The length of the dataset: 26719
08/08 18:53:31 - mmengine - INFO - The number of instances per category in the dataset:
+----------------------+--------+
| category             | number |
+----------------------+--------+
| car                  | 105060 |
| truck                | 18913  |
| construction_vehicle | 3504   |
| bus                  | 4189   |
| trailer              | 4970   |
| motorcycle           | 2226   |
| bicycle              | 2303   |
| pedestrian           | 51121  |
+----------------------+--------+


In [11]:
from projects.detr3d.vis_zlt import visualizer_zlt
from projects.detr3d.detr3d import DETR3D
import os.path as osp
import numpy as np
from projects.detr3d.detr3d import DETR3D
from mmdet3d.structures import LiDARInstance3DBoxes
from mmengine.structures import InstanceData
detr3d = DETR3D()
vis = visualizer_zlt(debug_name='',vis_count=10000)


In [ ]:
tmp = dataset[930]

In [ ]:
print(tmp['data_samples'].gt_instances_3d)

In [ ]:
tmp['data_samples'].metainfo['lidar_path']

In [ ]:
import torch
sample_idx = '0a0d6b8c2e884134a3b48df43d54c36a'
trans = [ 646.2525, 1612.6713,    1.8371]# should be lidar2ego but not 2global
kpts = np.load('/home/zhenglt/mmdev11/detr3d-dev11/debug/0a0d6b8c2e884134a3b48df43d54c36a.npy')
kpts_3d = np.zeros((kpts.shape[0],7))
gt_labels_3d = torch.zeros((kpts.shape[0]))
kpts_3d[:,:2] = kpts
kpts_3d[:,2] -= trans[-1]
lidar_pts = np.load('/home/zhenglt/mmdev11/detr3d-dev11/debug/0a0d6b8c2e884134a3b48df43d54c36a.bin.npy')

In [ ]:
gt_bboxes_3d = LiDARInstance3DBoxes(torch.tensor(kpts_3d),box_dim=7,origin=(0.5,0.5,0.5))
map_inst = vis.toInstance({'gt_bboxes_3d': gt_bboxes_3d, 'gt_labels_3d': gt_labels_3d}, device=gt_labels_3d.device)

In [12]:
map_inst = None
for i in range(30,31,1):
    frame = dataset[i]
    name = osp.basename(frame['data_samples'].lidar_path).split('.')[0]
    scene = frame['data_samples'].city_name
    inst = frame['data_samples'].gt_instances_3d[:2]
    batch_data_samples = frame['data_samples']
    batch_input_metas = [batch_data_samples.metainfo]
    batch_input_metas = detr3d.add_lidar2img(batch_input_metas)
    gt_bboxes_3d = inst.bboxes_3d
    corners_pt = np.array(gt_bboxes_3d.corners.view(-1, 3)).T
    print(name)
    # vis.visualize(inst, batch_input_metas,None,'test','debug/')
    # cat_instances = InstanceData.cat([map_inst, vis.add_score(frame['data_samples'].gt_instances_3d)])
    vis.visualize_dataset_item(frame,[vis.add_score(frame['data_samples'].gt_instances_3d)],pts = None,name_suffix='new_dataset', dirname='debug/dataset_op')

n015-2018-07-18-11-07-57+0800__LIDAR_TOP__1531883545549635
singapore-onenorth


[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
reference torch.Size([3, 3])
900 1600
torch.Size([1, 2, 3])
frame: 30


Compose(
    maybe we need to fix this bug
    LoadMultiViewImageFromFiles(to_float32=True, color_type='unchanged', num_views=1, num_ref_frames=-1, test_mode=False)
    maybe we need to fix this bug
    LoadAnnotations3D(
    with_bbox_3d=True,     with_label_3d=True,     with_attr_label=False,     with_mask_3d=False,     with_seg_3d=False,     with_panoptic_3d=False,     with_bbox=False,     with_label=False,     with_mask=False,     with_seg=False,     with_bbox_depth=False,     poly2mask=True)    seg_offset=None)
    ObjectNameFilter(classes=['car', 'truck', 'construction_vehicle', 'bus', 'trailer', 'motorcycle', 'bicycle', 'pedestrian'])
    ObjectRangeFilter(point_cloud_range=[0.0, -51.20000076293945, -5.0, 51.20000076293945, 51.20000076293945, 4.0])
    ProjectLabelToWaymoClass
    Pack3DDetInputsExtra(keys=['img', 'gt_bboxes_3d', 'gt_labels_3d'])(meta_keys=('img_path', 'ori_shape', 'img_shape', 'lidar2img', 'depth2img', 'cam2img', 'pad_shape', 'scale_factor', 'flip', 'pcd_horizo

In [ ]:
map_inst = None
for i in range(930,931,1):
    frame = dataset[i]
    name = osp.basename(frame['data_samples'].lidar_path).split('.')[0]
    scene = frame['data_samples'].city_name
    inst = frame['data_samples'].gt_instances_3d[:2]
    batch_data_samples = frame['data_samples']
    batch_input_metas = [batch_data_samples.metainfo]
    batch_input_metas = detr3d.add_lidar2img(batch_input_metas)
    gt_bboxes_3d = inst.bboxes_3d
    corners_pt = np.array(gt_bboxes_3d.corners.view(-1, 3)).T
    print(name)
    # vis.visualize(inst, batch_input_metas,None,'test','debug/')
    # cat_instances = InstanceData.cat([map_inst, vis.add_score(frame['data_samples'].gt_instances_3d)])
    vis.visualize_dataset_item(frame,[vis.add_score(frame['data_samples'].gt_instances_3d)],pts = None,name_suffix='small_pcrange', dirname='debug/lyc_map')

In [ ]:
# token2sample_idx
import pandas
import mmengine
infos = pandas.read_pickle('data/nus_v2/nuscenes_infos_trainval.pkl')



In [ ]:
token2sampleidx = {}
for i,info in enumerate(infos['data_list']):
    token2sampleidx[info['token']] = i
mmengine.dump(token2sampleidx,'debug/lyc_map/token2idx_trainval.pkl')